In [1]:
sc

In [2]:
sc.version

'1.6.0'

In [3]:
sc.appName

'pyspark-shell'

In [5]:
!ls /resources/data/TamingBigDataPySpark/ml-100k

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


# Exploring 100k Movielens Dataset

In [21]:
lines = sc.textFile("/resources/data/TamingBigDataPySpark//ml-100k/u.data",4)
lines.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [22]:
lines.getNumPartitions()

4

In [18]:
lines.map(lambda x: x.split())

False

In [25]:
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()
result

defaultdict(int, {'1': 6110, '2': 11370, '3': 27145, '4': 34174, '5': 21201})

In [26]:
print("Histogram of Ratings")
for key, value in sorted(result.items()):
    print(key, value)

Histogram of Ratings
1 6110
2 11370
3 27145
4 34174
5 21201


# Explore FakeFriends.csv

In [27]:
a = sc.textFile("/resources/data/TamingBigDataPySpark/fakefriends.csv")
a.take(5)

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [28]:
b = a.map(lambda x:x.split(","))
b.take(5)

[['0', 'Will', '33', '385'],
 ['1', 'Jean-Luc', '26', '2'],
 ['2', 'Hugh', '55', '221'],
 ['3', 'Deanna', '40', '465'],
 ['4', 'Quark', '68', '21']]

In [30]:
a.flatMap(lambda x:x.split(",")).take(20)

['0',
 'Will',
 '33',
 '385',
 '1',
 'Jean-Luc',
 '26',
 '2',
 '2',
 'Hugh',
 '55',
 '221',
 '3',
 'Deanna',
 '40',
 '465',
 '4',
 'Quark',
 '68',
 '21']

In [29]:
c = b.map(lambda x: (int(x[2]), int(x[3])))
c.take(5)

[(33, 385), (26, 2), (55, 221), (40, 465), (68, 21)]

In [34]:
d = c.mapValues(lambda x: (x,1))
d.collect()

[(33, (385, 1)),
 (26, (2, 1)),
 (55, (221, 1)),
 (40, (465, 1)),
 (68, (21, 1)),
 (59, (318, 1)),
 (37, (220, 1)),
 (54, (307, 1)),
 (38, (380, 1)),
 (27, (181, 1)),
 (53, (191, 1)),
 (57, (372, 1)),
 (54, (253, 1)),
 (56, (444, 1)),
 (43, (49, 1)),
 (36, (49, 1)),
 (22, (323, 1)),
 (35, (13, 1)),
 (45, (455, 1)),
 (60, (246, 1)),
 (67, (220, 1)),
 (19, (268, 1)),
 (30, (72, 1)),
 (51, (271, 1)),
 (25, (1, 1)),
 (21, (445, 1)),
 (22, (100, 1)),
 (42, (363, 1)),
 (49, (476, 1)),
 (48, (364, 1)),
 (50, (175, 1)),
 (39, (161, 1)),
 (26, (281, 1)),
 (53, (197, 1)),
 (43, (249, 1)),
 (27, (305, 1)),
 (32, (81, 1)),
 (58, (21, 1)),
 (64, (65, 1)),
 (31, (192, 1)),
 (52, (413, 1)),
 (67, (167, 1)),
 (54, (75, 1)),
 (58, (345, 1)),
 (35, (244, 1)),
 (52, (77, 1)),
 (25, (96, 1)),
 (24, (49, 1)),
 (20, (1, 1)),
 (40, (254, 1)),
 (51, (283, 1)),
 (36, (212, 1)),
 (19, (269, 1)),
 (62, (31, 1)),
 (19, (5, 1)),
 (41, (278, 1)),
 (44, (194, 1)),
 (57, (294, 1)),
 (59, (158, 1)),
 (59, (284, 1)),
 

In [35]:
e = d.reduceByKey(lambda v1, v2: (v1[0] + v2[0], v1[1] + v1[1]))
e.take(5)

[(64, (3376, 32)),
 (66, (2488, 32)),
 (68, (2696, 32)),
 (18, (2747, 4)),
 (20, (825, 8))]

In [36]:
f = e.mapValues(lambda x: x[0]/x[1])
f.take(5)

[(64, 105.5), (66, 77.75), (68, 84.25), (18, 686.75), (20, 103.125)]

# Find Min Temperature

In [37]:
!cat  "/resources/data/TamingBigDataPySpark/1800.csv" | head -5

ITE00100554,18000101,TMAX,-75,,,E,
ITE00100554,18000101,TMIN,-148,,,E,
GM000010962,18000101,PRCP,0,,,E,
EZE00100082,18000101,TMAX,-86,,,E,
EZE00100082,18000101,TMIN,-135,,,E,


In [38]:
inputFile = "/resources/data/TamingBigDataPySpark/1800.csv"
def parseLines(line):
    fields = line.split(",")
    #stationId, readingType, temperature = fields[0] , fields[1], float(fields[3]*0.1*(9/5)+32)
    return (fields[0] , fields[2], float(fields[3])*0.1*(9/5)+32)

def filterMinTemps(line):
    if "TMIN" in line:
        return line
    
inputRDD = sc.textFile(inputFile).map(parseLines)

minTempRDD = inputRDD.filter(filterMinTemps)

minTempPairRDD = minTempRDD.map(lambda x: (x[0], x[2]))

minTempResults = minTempPairRDD.reduceByKey(lambda x, y : x if x < y else y)

minTempResults.take(10)


[('ITE00100554', 5.359999999999999), ('EZE00100082', 7.699999999999999)]

In [58]:
def filterMaxTemps(line):
    if "TMAX" in line:
        return line
    
maxTempRDD = inputRDD.filter(filterMaxTemps)

maxTempPairRDD = maxTempRDD.map(lambda x: (x[0], x[2]))

maxTempResults = maxTempPairRDD.reduceByKey(lambda x, y : x if x > y else y)

maxTempResults.take(10)    

[('ITE00100554', 90.14000000000001), ('EZE00100082', 90.14000000000001)]

# Word Count

In [78]:
import string
def cleanFile(line):
    remove_punc_map = dict.fromkeys(map(lambda x: ord(x), string.punctuation),None)
    return (line.translate(remove_punc_map).lower().split())

a = sc.textFile("/resources/data/TamingBigDataPySpark/book.txt")
b = a.flatMap(cleanFile).countByValue()

for k,v in b.items():
    k = k.encode('ascii', 'ignore')
    print (k, v)


b'unsolved' 1
b'hitting' 1
b'paycheck' 11
b'picked' 2
b'precise' 2
b'phone' 9
b'stressful' 1
b'update' 4
b'40000' 2
b'happen' 13
b'street' 2
b'engineered' 1
b'institution' 1
b'inaction' 1
b'big' 40
b'moment' 3
b'capabilities' 1
b'impact' 4
b'obvious' 3
b'opengl' 1
b'park' 2
b'viewers' 1
b'reserve' 1
b'opposite' 1
b'happening' 1
b'trial' 4
b'clicked' 2
b'designed' 7
b'depressions' 1
b'rendering' 1
b'selected' 1
b'hell' 1
b'startup' 9
b'dump' 1
b'underneath' 2
b'foam' 1
b'stress' 1
b'intentionally' 2
b'also' 91
b'degree' 4
b'feature' 2
b'fares' 1
b'unproven' 1
b'suspect' 1
b'aerospace' 2
b'ultimately' 9
b'basic' 6
b'bother' 2
b'permits' 3
b'refines' 1
b'thousands' 7
b'93' 1
b'were' 47
b'leave' 8
b'trade' 7
b'too' 27
b'disable' 1
b'guts' 1
b'elses' 3
b'complement' 1
b'morning' 4
b'proxy' 2
b'bots' 1
b'placing' 1
b'system' 9
b'confidence' 1
b'sink' 1
b'misunderstandings' 1
b'contained' 1
b'ready' 3
b'relocation' 1
b'modern' 4
b'director' 1
b'lots' 7
b'helpareportercom' 1
b'extra' 21
b'tric

# Spark Advanced Examples
## 1. Find Most popular Movie in Movie Lens DataSet

In [113]:
result = (sc.textFile("/resources/data/TamingBigDataPySpark/ml-100k/u.data")
       .map(lambda x: (int(x.split()[1]),1))
       .reduceByKey(lambda x,y :x+y)
       .sortBy(keyfunc=lambda x: -x[1])
       .take(1)
      )  
for i in result:
    print("Movie {0} is most popular with {1} ratings in ml-100k set".format(i[0], i[1]))

Movie 50 is most popular with 583 ratings in ml-100k set


## 2. Use Broadcast Variables to Display Movie Names Instead of ID Numbers

In [1]:
#def loadMovieNames():
movieNames = {}
with open("/resources/data/TamingBigDataPySpark/ml-100k/u.item", encoding = "ISO-8859-1") as f:
    for line in f:
        fields = line.split("|")
        movieNames[int(fields[0])] = fields[1]
#    return movieNames

movieBroadCast = sc.broadcast(movieNames)

result = (sc.textFile("/resources/data/TamingBigDataPySpark/ml-100k/u.data")
          .map(lambda x: (int(x.split()[1]),1))
          .reduceByKey(lambda x,y :x+y)
          .sortBy(keyfunc=lambda x: -x[1])
          .map(lambda i : (movieBroadCast.value[i[0]], i[1]))
          .take(1)
      )  
for i in result:
    print("{0} is most popular movie with {1} ratings in ml-100k movie dataset".format(i[0], i[1]))

Star Wars (1977) is most popular movie with 583 ratings in ml-100k movie dataset


## 3. Find the Most Popular Superhero in a Social Graph

In [86]:
heroNames = {}
with open("/resources/data/TamingBigDataPySpark/Marvel-names.txt", encoding = "ISO-8859-1") as f:
    for line in f:
        fields = line.split()
        heroid = int(fields[0])
        heroName = " ".join([i.strip('"') for i in fields[1:]])
        heroNames[heroid] = heroName

movieBroadCast = sc.broadcast(heroNames)
result = (sc.textFile("/resources/data/TamingBigDataPySpark/Marvel-graph.txt")
          .map(lambda x: x.split())
          .map(lambda x: (int(x[0]),len(x[1:])))
          .reduceByKey(lambda x,y :x+y)
          .map(lambda x : (movieBroadCast.value[x[0]], x[1]))
          .sortBy(keyfunc=lambda x : -x[1])
      )  
result.take(5)

[('CAPTAIN AMERICA', 1933),
 ('SPIDER-MAN/PETER PAR', 1741),
 ('IRON MAN/TONY STARK ', 1528),
 ('THING/BENJAMIN J. GR', 1426),
 ('WOLVERINE/LOGAN ', 1394)]

## 4. Degrees of Seperation


In [169]:
inputFile = sc.textFile("/resources/data/TamingBigDataPySpark/Marvel-graph.txt")

In [170]:
# The characters we wish to find the degree of separation between:
startCharacterID = 5306 #SpiderMan
targetCharacterID = 14  #ADAM 3,031 (who?)
hitCounter = sc.accumulator(0)

In [173]:
def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = 'WHITE'
    distance = 9999

    if (heroID == startCharacterID):
        color = 'GRAY'
        distance = 0

    return (heroID, (connections, distance, color))

In [172]:
def bfsMap(node):
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]

    results = []

    #If this node needs to be expanded...
    if (color == 'GRAY'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = 'GRAY'
            if (targetCharacterID == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)

        #We've processed this node, so color it black
        color = 'BLACK'

    #Emit the input node so we don't lose it.
    results.append( (characterID, (connections, distance, color)) )
    return results

In [174]:
mapped = iterationRdd.flatMap(bfsMap)
for i in mapped.collect():
    if i[1][2] == 'GRAY':
        print(i)

(1718, ([], 1, 'GRAY'))
(4024, ([], 1, 'GRAY'))
(5981, ([], 1, 'GRAY'))
(5986, ([], 1, 'GRAY'))
(5984, ([], 1, 'GRAY'))
(340, ([], 1, 'GRAY'))
(3998, ([], 1, 'GRAY'))
(3995, ([], 1, 'GRAY'))
(3994, ([], 1, 'GRAY'))
(349, ([], 1, 'GRAY'))
(3990, ([], 1, 'GRAY'))
(6356, ([], 1, 'GRAY'))
(967, ([], 1, 'GRAY'))
(4933, ([], 1, 'GRAY'))
(960, ([], 1, 'GRAY'))
(4733, ([], 1, 'GRAY'))
(4731, ([], 1, 'GRAY'))
(2315, ([], 1, 'GRAY'))
(5855, ([], 1, 'GRAY'))
(5858, ([], 1, 'GRAY'))
(293, ([], 1, 'GRAY'))
(291, ([], 1, 'GRAY'))
(3771, ([], 1, 'GRAY'))
(3770, ([], 1, 'GRAY'))
(3777, ([], 1, 'GRAY'))
(3775, ([], 1, 'GRAY'))
(968, ([], 1, 'GRAY'))
(3779, ([], 1, 'GRAY'))
(4522, ([], 1, 'GRAY'))
(270, ([], 1, 'GRAY'))
(272, ([], 1, 'GRAY'))
(273, ([], 1, 'GRAY'))
(274, ([], 1, 'GRAY'))
(4650, ([], 1, 'GRAY'))
(278, ([], 1, 'GRAY'))
(279, ([], 1, 'GRAY'))
(4658, ([], 1, 'GRAY'))
(4660, ([], 1, 'GRAY'))
(1780, ([], 1, 'GRAY'))
(2262, ([], 1, 'GRAY'))
(2266, ([], 1, 'GRAY'))
(2446, ([], 1, 'GRAY'))
(2449

In [167]:
a = sc.parallelize([[(1718, ([], 1, 'GRAY')), (4024, ([], 1, 'GRAY'))], [(991, ([], 1, 'GRAY')), (4617, ([], 1, 'GRAY'))]])

In [168]:
a.collect()

[[(1718, ([], 1, 'GRAY')), (4024, ([], 1, 'GRAY'))],
 [(991, ([], 1, 'GRAY')), (4617, ([], 1, 'GRAY'))]]

In [151]:
b = a.map(lambda x:(x,1))
b.collect()

[([(1718, ([], 1, 'GRAY')), (4024, ([], 1, 'GRAY'))], 1),
 ([(991, ([], 1, 'GRAY')), (4617, ([], 1, 'GRAY'))], 1)]

In [154]:
c = a.flatMap(lambda x:(x[0],x[1]))
c.collect()

[(1718, ([], 1, 'GRAY')),
 (4024, ([], 1, 'GRAY')),
 (991, ([], 1, 'GRAY')),
 (4617, ([], 1, 'GRAY'))]

In [164]:
line1 = "Jane Mulkerrins"
line2 = "money and"
line3 ="sandy"
rdd = sc.parallelize([line1, line2, line3])

In [165]:
rdd.map(lambda x: x.split()).collect()

[['Jane', 'Mulkerrins'], ['money', 'and'], ['sandy']]

In [166]:
rdd.flatMap(lambda x: x.split()).collect()

['Jane', 'Mulkerrins', 'money', 'and', 'sandy']

In [115]:
for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))

    # Create new vertices as needed to darken or reduce distances in the
    # reduce stage. If we encounter the node we're looking for as a GRAY
    # node, increment our accumulator to signal that we're done.
    mapped = iterationRdd.flatMap(bfsMap)

    # Note that mapped.count() action here forces the RDD to be evaluated, and
    # that's the only reason our accumulator is actually updated.
    print("Processing " + str(mapped.count()) + " values.")

    if (hitCounter.value > 0):
        print("Hit the target character! From " + str(hitCounter.value) \
            + " different direction(s).")
        break

    # Reducer combines data for each character ID, preserving the darkest
    # color and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)                          ]

['WHITE']

In [109]:
for i in iterationRdd.collect():
    if i[1][2] == 'GRAY':
        print(i)

(5306, ([1718, 4024, 5981, 5986, 5984, 340, 3998, 3995, 3994, 349, 3990, 6356, 967, 4933, 960, 4733, 4731, 2315, 5855, 5858, 293, 291, 3771, 3770, 3777, 3775, 968, 3779, 4522, 270, 272, 273, 274, 4650, 278, 279, 4658, 4660, 1780, 2262, 2266, 2446, 2449, 107, 2046, 2045, 2040, 2041, 2048, 4145, 6372, 3519, 3517, 3515, 6206, 6207, 2689, 6202, 6203, 2686, 6208, 99, 98, 91, 1623, 5183, 3205, 1992, 1994, 1628, 3209, 3208, 2865, 555, 557, 5480, 5458, 3341, 5455, 1197, 1196, 1756, 1750, 1177, 1175, 1173, 4253, 4252, 5533, 5532, 5537, 5535, 1005, 1287, 1286, 1289, 689, 3431, 685, 1572, 1571, 686, 458, 620, 4383, 4381, 5622, 4384, 1370, 405, 403, 1376, 3834, 3835, 3836, 3837, 3832, 4033, 5996, 5999, 377, 392, 6141, 6140, 6144, 399, 398, 2309, 2303, 4726, 4721, 4722, 5846, 5843, 5842, 5841, 5840, 5849, 3748, 3740, 240, 4629, 2275, 2273, 2279, 2459, 2450, 2453, 179, 178, 2052, 2055, 2054, 2057, 2056, 3690, 3691, 4530, 3696, 6367, 6366, 6365, 3524, 3523, 3520, 3086, 6217, 6216, 6214, 6211, 2698, 4

In [42]:
f.readline()

'3 "4-D MAN/MERCURIO"\n'

In [44]:
for line in f:
    print(line)

1 "24-HOUR MAN/EMMANUEL"

2 "3-D MAN/CHARLES CHAN"

3 "4-D MAN/MERCURIO"

4 "8-BALL/"

5 "A"

6 "A'YIN"

7 "ABBOTT, JACK"

8 "ABCISSA"

9 "ABEL"

10 "ABOMINATION/EMIL BLO"

11 "ABOMINATION | MUTANT"

12 "ABOMINATRIX"

13 "ABRAXAS"

14 "ADAM 3,031"

15 "ABSALOM"

16 "ABSORBING MAN/CARL C"

17 "ABSORBING MAN | MUTA"

18 "ACBA"

19 "ACHEBE, REVEREND DOC"

20 "ACHILLES"

21 "ACHILLES II/HELMUT"

22 "ACROBAT/CARL ZANTE"

23 "ADAM X"

24 "ADAMS, CINDY"

25 "ADAMS, CONGRESSMAN H"

26 "ADAMS, GEORGE"

27 "ADAMS, MARTHA"

28 "ADAMS, NICOLE NIKKI"

29 "ADAMSON, JASON"

30 "ADAMSON, REBECCA"

31 "ADMIRAL PROTOCOL/"

32 "ADORA"

33 "ADORA CLONE"

34 "ADRIA"

35 "ADVA"

36 "ADVENT/KYLE GROBE"

37 "ADVERSARY"

38 "AEGIS/TREY ROLLINS"

39 "AENTAROS"

40 "AFTERLIFE"

41 "AGAMEMNON"

42 "AGAMEMNON II/ANDREI "

43 "AGAMEMNON III/"

44 "AGAMOTTO"

45 "AGARN, CAPT."

46 "AGED GENGHIS"

47 "AGEE, DR. AUBREY"

48 "AGEE, REBECCA"

49 "AGENT AXIS/"

50 "AGENT 18/JACK TRUMAN"

51 "AGENT-X/"

52 "AGGAMON"

53 "

In [45]:
f.closed

False